<a href="https://colab.research.google.com/github/p4zaa/OTTO-Multi-Objective-Recommender-System/blob/main/%5BDEVELOP1_7%5D_OTTO_%E2%80%93_Multi_Objective_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Version Logs
* [View in my Github](https://github.com/p4zaa/OTTO-Multi-Objective-Recommender-System)

# Import Libraries

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
import pickle

In [20]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install -q torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-geometric

In [21]:
import pandas as pd, numpy as np
from itertools import product
import io, os, json
import tqdm

import time

from sklearn.metrics import mean_squared_error

#import plotly.io as pio
#import plotly.express as px
#import plotly.graph_objects as go
#from plotly.subplots import make_subplots
#pio.templates.default = "plotly_white"

import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import MovieLens
from torch_geometric.nn import to_hetero
from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv, GATConv, Linear, GraphConv
from torch_geometric.data import Data, HeteroData
from torch_geometric.loader import HGTLoader, NeighborLoader, LinkNeighborLoader

!pip install -q sentence-transformers

%matplotlib inline

In [22]:
!pip3 install -q torch torchvision torchaudio
!pip install -U polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
import polars as pl

In [24]:
print('Pytorch CUDA Version is ', torch.version.cuda)

Pytorch CUDA Version is  11.6


In [25]:
torch.cuda.empty_cache()

In [26]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jan  3 09:14:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [27]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [28]:
#!nvidia-smi

In [29]:
torch.cuda.is_available()

True

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load Competition Dataset

In [31]:
'''
from google.colab import files
files.upload() # expire any previous token(s) and upload recreated token
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets list
'''

'\nfrom google.colab import files\nfiles.upload() # expire any previous token(s) and upload recreated token\n!rm -r ~/.kaggle\n!mkdir ~/.kaggle\n!mv ./kaggle.json ~/.kaggle/\n!chmod 600 ~/.kaggle/kaggle.json\n\n!kaggle datasets list\n'

In [32]:
#!kaggle datasets download 'radek1/otto-full-optimized-memory-footprint' -p /content/kaggle/ --unzip

In [33]:
#!kaggle datasets download 'radek1/otto-train-and-test-data-for-local-validation' -p /content/dataset/ --unzip

## Files
- **train.jsonl** - the training data, which contains full session data
  * `session` - the unique session id
  * `events` - the time ordered sequence of events in the session
    * `aid` - the article id (product code) of the associated event
    * `ts` - the Unix timestamp of the event
    * `type` - the event type, i.e., whether a product was clicked, added to the user's cart, or ordered during the session
###### {'clicks': 0, 'carts': 1, 'orders': 2}
- **test.jsonl** - the test data, which contains truncated session data
  * your task is to predict the next `aid` clicked after the session truncation, as well as the the remaining `aids` that are added to `carts` and `orders`; you may predict up to 20 values for each session `type`
- **sample_submission.csv** - a sample submission file in the correct format

In [34]:
# Pandas
#df = pd.read_parquet('/content/kaggle/train.parquet')

# Polars
#df = pl.read_parquet('/content/drive/MyDrive/OTTO-Kaggle-Competition/dataset/train.parquet')

In [35]:
# sample 10%(frac=0.1) of data
#df = df.sample(frac=0.01, replace=False)

# Polars
#df = df.sample(frac=0.01)

In [36]:
# Save 1% dataset
#df.write_parquet('/content/drive/MyDrive/OTTO-Kaggle-Competition/train_frac0.01.parquet')

# Load
#df = pl.read_parquet('/content/drive/MyDrive/OTTO-Kaggle-Competition/train_frac0.01.parquet')

In [37]:
df = df.to_pandas()

In [38]:
df

,session,aid,ts,type
0,9841405,863362,1661168453,0
1,748820,781589,1660026965,0
2,2910988,1213780,1659556275,0
3,6064205,927013,1660241721,0
4,8863318,284676,1661065123,0
...,...,...,...,...
2167155,398294,1186719,1660991442,1
2167156,913121,524506,1659613897,0
2167157,97582,118319,1660665916,0
2167158,4336715,283656,1661192088,0


In [39]:
#df.isna().sum()

In [40]:
df['type'].unique()

array([0, 1, 2], dtype=uint8)

In [41]:
#df.info()

# Construct Graph Data (memory-efficient optimized)

### `edge_index` new code construct

In [42]:
# session index dict
session = df['session'].unique()
source_idx = {id:idx for idx, id in enumerate(session)}

# aid(article id) index dict
aid = df['aid'].unique()
target_idx = {id:idx for idx, id in enumerate(aid)}

# Load data (deserialize)
#with open('/content/drive/MyDrive/OTTO-Kaggle-Competition/source_idx.pkl', 'rb') as file:
#    source_idx = pickle.load(file)

#with open('/content/drive/MyDrive/OTTO-Kaggle-Competition/target_idx.pkl', 'rb') as file:
#    target_idx = pickle.load(file)

In [43]:
connected = df[['session', 'aid']]
connected['session'] = connected['session'].map(source_idx)
connected['aid'] = connected['aid'].map(target_idx)

<ipython-input-43-fdbbf2e7eb54>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  connected['session'] = connected['session'].map(source_idx)
<ipython-input-43-fdbbf2e7eb54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  connected['aid'] = connected['aid'].map(target_idx)


In [44]:
source = connected['session']
target = connected['aid']
edge_index = torch.tensor((source.values, target.values))

<ipython-input-44-23961f25841f>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  edge_index = torch.tensor((source.values, target.values))


In [45]:
del connected
gc.collect()

35

### Nodes and Edges Attribute

In [46]:
df['session']

0          9841405
1           748820
2          2910988
3          6064205
4          8863318
            ...   
2167155     398294
2167156     913121
2167157      97582
2167158    4336715
2167159    5517556
Name: session, Length: 2167160, dtype: int32

In [47]:
## Nodes Atrributes
session_num_nodes = df['session'].nunique()
aid_num_nodes = df['aid'].nunique()
session_id = torch.tensor(list(source_idx.values()), dtype=torch.int64)

aid_features = torch.rand((aid_num_nodes, 32)) # Create (random) article features with shape [num_node_aid, dimensions]
aid_id = torch.tensor(list(target_idx.values()), dtype=torch.int64)
#aid_features = torch.nn.Embedding(aid_num_nodes, 32)

## Edges Atrributes
edge_index = edge_index
edge_label = torch.tensor(df['type'].values).type(torch.int64)

In [48]:
del df
gc.collect()

0

### Construct HeteroData

In [49]:
node_types = {
    'session': {
        #'num_nodes': session_num_nodes,
        'node_id': session_id
    },
    'aid': {
        'x': aid_features,
        'node_id': aid_id
    }
}

edge_types = {
    ('session', 'event', 'aid'): {
        'edge_index': edge_index,
        'edge_label': edge_label
    }#,
    #('session', 'cart', 'aid'): {
        
    #},
    #('session', 'buy', 'aid'): {
        
    #}
}

In [50]:
data = HeteroData({**node_types, **edge_types})

In [51]:
data

HeteroData(
  session={ node_id=[1607917] },
  aid={
    x=[569367, 32],
    node_id=[569367]
  },
  (session, event, aid)={
    edge_index=[2, 2167160],
    edge_label=[2167160]
  }
)

In [52]:
data.metadata()

(['session', 'aid'], [('session', 'event', 'aid')])

In [53]:
del source, target, edge_index, edge_label
gc.collect()

0

In [54]:
node_types, edge_types = data.metadata()
print('Node types:', node_types)
print('Edge types:',edge_types)

Node types: ['session', 'aid']
Edge types: [('session', 'event', 'aid')]


In [55]:
#print('Isolated nodes?', data.has_isolated_nodes())
#print('Self loops?', data.has_self_loops())
#print('Undirected graph? ', data.is_undirected())

# Graph-based Modeling [follow [this](https://github.com/pyg-team/pytorch_geometric/blob/master/examples/hetero/hetero_link_pred.py) sample]

In [56]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Construct Undirected Graph

In [57]:
# add sesion features for message passing:
data['session'].x = torch.rand(data['session'].num_nodes, 32)
#data['session'].x = torch.nn.Embedding(data['session'].num_nodes, 32)

In [58]:
# Add user node features for message passing:
#data['session'].x = torch.eye(data['session'].num_nodes, device=device)
#del data['session'].num_nodes

In [59]:
# Add a reverse ('movie', 'rev_rates', 'user') relation for message passing:
data = T.ToUndirected()(data)

In [60]:
del data['aid', 'rev_event', 'session'].edge_label  # Remove "reverse" label.

In [61]:
data

HeteroData(
  session={
    node_id=[1607917],
    x=[1607917, 32]
  },
  aid={
    x=[569367, 32],
    node_id=[569367]
  },
  (session, event, aid)={
    edge_index=[2, 2167160],
    edge_label=[2167160]
  },
  (aid, rev_event, session)={ edge_index=[2, 2167160] }
)

### [Follow [this sample](https://github.com/pyg-team/pytorch_geometric/blob/master/examples/hetero/hetero_link_pred.py)] Train/Val/Test Link Level Splits

In [62]:
# Perform a link-level split into training, validation, and test edges:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    neg_sampling_ratio=0.0,
    edge_types=[('session', 'event', 'aid')],
    rev_edge_types=[('aid', 'rev_event', 'session')],
)(data)

### Mini Batch

In [63]:
# Define seed edges:
edge_label_index = train_data['session', 'event', 'aid'].edge_label_index
edge_label = train_data['session', 'event', 'aid'].edge_label

train_loader = LinkNeighborLoader(
    data=train_data,  # TODO
    num_neighbors=[5, 2],  # TODO
    neg_sampling_ratio=0.0,  # TODO
    edge_label_index=(('session', 'event', 'aid'), edge_label_index),
    edge_label=edge_label,
    batch_size=64,
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

#assert sampled_data["session", "event", "aid"].edge_label_index.size(1) == 128
#assert sampled_data["session", "event", "aid"].edge_label.min() == 0
#assert sampled_data["session", "event", "aid"].edge_label.max() == 2

Sampled mini-batch:
HeteroData(
  session={
    node_id=[336],
    x=[336, 32]
  },
  aid={
    x=[181, 32],
    node_id=[181]
  },
  (session, event, aid)={
    edge_index=[2, 317],
    edge_label=[64],
    edge_label_index=[2, 64],
    input_id=[64]
  },
  (aid, rev_event, session)={ edge_index=[2, 392] }
)


### [New weight calculation code]

In [64]:
# We have an unbalanced dataset with many labels for rating 3 and 4, and very
# few for 0 and 1. Therefore we use a weighted MSE loss.
weight = torch.bincount(train_data['session', 'aid'].edge_label)
weight = weight.max() / weight

In [65]:
weight

tensor([ 1.0000, 11.5327, 38.0288])

### Model and loss functions

#### Loss function

In [66]:
def weighted_mse_loss(pred, target, weight=None):
    weight = 1. if weight is None else weight[target].to(pred.dtype)
    return (weight * (pred - target.to(pred.dtype)).pow(2)).mean()

#### Model

In [67]:
'''
class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.layers = torch.nn.ModuleList([
            SAGEConv((-1, -1), hidden_channels),
            SAGEConv((-1, -1), out_channels)
        ])

    def forward(self, x, edge_index):
        for layer in self.layers:
            x = layer(x, edge_index)
        return x.relu()

class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(2 * hidden_channels, hidden_channels),
            torch.nn.BatchNorm1d(hidden_channels),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_channels, 1)
        )

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['session'][row], z_dict['aid'][col]], dim=-1)
        return self.layers(z).view(-1)

class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # encoder and decoder
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

        # embedding matrices for sessions and aids
        self.aid_lin = torch.nn.Linear(hidden_channels, hidden_channels)
        self.session_emb = torch.nn.Embedding(data['session'].num_nodes, hidden_channels)
        self.aid_emb = torch.nn.Embedding(data['aid'].num_nodes, hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        #x_dict = {
        #    'session': self.session_emb(data['session'].node_id),
        #    'aid': self.aid_lin(data['aid'].x.float()) + self.aid_emb(data['aid'].node_id)
        #}
        #x_dict = x_dict.copy()
        #x_dict['session'] = self.session_emb.weight
        #x_dict['aid'] = self.aid_emb.weight
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)
'''


"\nclass GNNEncoder(torch.nn.Module):\n    def __init__(self, hidden_channels, out_channels):\n        super().__init__()\n        self.layers = torch.nn.ModuleList([\n            SAGEConv((-1, -1), hidden_channels),\n            SAGEConv((-1, -1), out_channels)\n        ])\n\n    def forward(self, x, edge_index):\n        for layer in self.layers:\n            x = layer(x, edge_index)\n        return x.relu()\n\nclass EdgeDecoder(torch.nn.Module):\n    def __init__(self, hidden_channels):\n        super().__init__()\n        self.layers = torch.nn.Sequential(\n            torch.nn.Linear(2 * hidden_channels, hidden_channels),\n            torch.nn.BatchNorm1d(hidden_channels),\n            torch.nn.ReLU(),\n            torch.nn.Linear(hidden_channels, 1)\n        )\n\n    def forward(self, z_dict, edge_label_index):\n        row, col = edge_label_index\n        z = torch.cat([z_dict['session'][row], z_dict['aid'][col]], dim=-1)\n        return self.layers(z).view(-1)\n\nclass Model(to

In [68]:

class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['session'][row], z_dict['aid'][col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z)

        return z.view(-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # encoder and decoder
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

        # embedding matrices for sessions and aids
        self.aid_lin = Linear(hidden_channels, hidden_channels)
        self.session_emb = torch.nn.Embedding(data['session'].num_nodes, hidden_channels)
        self.aid_emb = torch.nn.Embedding(data['aid'].num_nodes, hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        x_dict = {
            'session': self.session_emb(data['session'].node_id.to(device)),
            'aid': self.aid_lin(data['aid'].x.to(device).float()) + self.aid_emb(data['aid'].node_id.to(device))
        }
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)


In [69]:
model = Model(hidden_channels=32).to(device)

In [70]:
# Due to lazy initialization, we need to run one model step so the number
# of parameters can be inferred:

sampled = next(iter(train_loader))

with torch.no_grad():
    model.encoder(sampled_data.to(device).x_dict, sampled_data.to(device).edge_index_dict)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [71]:
del sampled, optimizer
gc.collect()

249

In [72]:
'''
def train():
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data['session', 'aid'].edge_label_index)
    target = train_data['session', 'aid'].edge_label
    loss = weighted_mse_loss(pred, target, weight)
    loss.backward()
    optimizer.step()
    return float(loss)
'''

"\ndef train():\n    model.train()\n    optimizer.zero_grad()\n    pred = model(train_data.x_dict, train_data.edge_index_dict,\n                 train_data['session', 'aid'].edge_label_index)\n    target = train_data['session', 'aid'].edge_label\n    loss = weighted_mse_loss(pred, target, weight)\n    loss.backward()\n    optimizer.step()\n    return float(loss)\n"

In [73]:
'''
@torch.no_grad()
def test(data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict,
                 data['session', 'aid'].edge_label_index)
    pred = pred.clamp(min=-1, max=2)
    target = data['session', 'aid'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse)
'''

"\n@torch.no_grad()\ndef test(data):\n    model.eval()\n    pred = model(data.x_dict, data.edge_index_dict,\n                 data['session', 'aid'].edge_label_index)\n    pred = pred.clamp(min=-1, max=2)\n    target = data['session', 'aid'].edge_label.float()\n    rmse = F.mse_loss(pred, target).sqrt()\n    return float(rmse)\n"

In [74]:

def train(train_data=sampled_data):
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data[('session', 'event', 'aid')].edge_label_index)
    target = train_data['session', 'aid'].edge_label
    loss = weighted_mse_loss(pred, target, weight.to(device))
    #loss = F.binary_cross_entropy_with_logits(pred, target)
    loss.backward()
    optimizer.step()
    return float(loss), pred, model


In [75]:
'''
def train(model, train_data):
    edge_index_dict, edge_label_index, edge_label = train_data.edge_index_dict, train_data.edge_label_index, train_data.edge_label

    pred = model(edge_index_dict, edge_label_index)
    loss = F.binary_cross_entropy_with_logits(pred, edge_label.to(device).float())
    loss.backward()
    optimizer.step()

    return loss.item(), pred, model
'''

'\ndef train(model, train_data):\n    edge_index_dict, edge_label_index, edge_label = train_data.edge_index_dict, train_data.edge_label_index, train_data.edge_label\n\n    pred = model(edge_index_dict, edge_label_index)\n    loss = F.binary_cross_entropy_with_logits(pred, edge_label.to(device).float())\n    loss.backward()\n    optimizer.step()\n\n    return loss.item(), pred, model\n'

In [76]:
@torch.no_grad()
def test(data=data):
    model.eval()
    #pred = model(data.x_dict, data.edge_index_dict,
    #             data['session', 'aid'].edge_label_index)
    pred = model(data.edge_index_dict,
                 data['session', 'aid'].edge_label_index)
    pred = pred.clamp(min=0, max=2)
    target = data['session', 'aid'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse), pred, target

In [77]:
@torch.no_grad()
def test(model=model, data=data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict,
                 data['session', 'aid'].edge_label_index)
    pred = pred.clamp(min=0, max=2)
    target = data['session', 'aid'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse), pred, target

### [mini-batch] Training Heterogenous Link-level GNN

In [78]:
data = data.to(device)

In [79]:
%%time
import torch.nn.functional as F

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

#model = Model(hidden_channels=32).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(0, 10):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):
        sampled_data = sampled_data.to(device)
        optimizer.zero_grad()
        loss, pred, model = train(train_data=sampled_data)
        #loss, pred, model = train(model=model, train_data=sampled_data)
        # TODO: Move `sampled_data` to the respective `device`
        # TODO: Run `forward` pass of the model
        # TODO: Apply binary cross entropy via
        # `F.binary_cross_entropy_with_logits(pred, ground_truth)`

        #loss.backward()
        #optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

Device: 'cuda'


  8%|▊         | 2151/27090 [04:54<56:53,  7.31it/s]


KeyboardInterrupt: ignored

### Save and load trained model

In [62]:
path = '/content/drive/MyDrive/OTTO-Kaggle-Competition/model_frac0.1.pt'

In [ ]:
# Save the entire model
torch.save(model, path)
model

In [53]:
# Load the entire model
trained_model = torch.load(path, map_location=torch.device('cpu'))
trained_model

Model(
  (encoder): GraphModule(
    (conv1): Module(
      (session__event__aid): SAGEConv((-1, -1), 32, aggr=mean)
      (aid__rev_event__session): SAGEConv((-1, -1), 32, aggr=mean)
    )
    (conv2): Module(
      (session__event__aid): SAGEConv((-1, -1), 32, aggr=mean)
      (aid__rev_event__session): SAGEConv((-1, -1), 32, aggr=mean)
    )
  )
  (decoder): EdgeDecoder(
    (lin1): Linear(64, 32, bias=True)
    (lin2): Linear(32, 1, bias=True)
  )
  (aid_lin): Linear(32, 32, bias=True)
  (session_emb): Embedding(208340, 32)
  (aid_emb): Embedding(127651, 32)
)

In [63]:
data.x_dict

{'session': tensor([[0.7716, 0.3446, 0.9334,  ..., 0.3412, 0.6989, 0.5248],
         [0.4549, 0.8710, 0.0269,  ..., 0.9187, 0.3608, 0.1913],
         [0.1990, 0.6910, 0.1618,  ..., 0.7928, 0.6775, 0.8326],
         ...,
         [0.7160, 0.7472, 0.6706,  ..., 0.9311, 0.3370, 0.0451],
         [0.4082, 0.7987, 0.1183,  ..., 0.8273, 0.5842, 0.6746],
         [0.9297, 0.5876, 0.8179,  ..., 0.4952, 0.7731, 0.3970]],
        device='cuda:0'),
 'aid': tensor([[0.0995, 0.3507, 0.7971,  ..., 0.3299, 0.2194, 0.7459],
         [0.2795, 0.7814, 0.0757,  ..., 0.8444, 0.3914, 0.3840],
         [0.4629, 0.0553, 0.8544,  ..., 0.9361, 0.5295, 0.2719],
         ...,
         [0.8227, 0.7462, 0.1603,  ..., 0.8294, 0.1937, 0.9485],
         [0.8318, 0.7501, 0.6972,  ..., 0.2137, 0.8887, 0.8755],
         [0.5684, 0.7075, 0.0961,  ..., 0.2217, 0.6406, 0.2281]],
        device='cuda:0')}

In [62]:
model.encoder(data.to(device).x_dict, data.to(device).edge_index_dict)

{'session': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0',
        grad_fn=<ReluBackward0>),
 'aid': tensor([[  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000, 285.4922],
         [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000, 290.7384],
         [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000, 263.1417],
         ...,
         [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000, 163.7994],
         [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000, 264.9494],
         [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000, 284.5202]],
        device='cuda:0', grad_fn=<ReluBackward0>)}

In [54]:
trained_model.encoder(data.to(device).x_dict, data.to(device).edge_index_dict)

{'session': tensor([[-1.3896e-01, -3.4696e-02,  1.7249e-01,  ...,  9.6917e-02,
           3.9357e-01, -2.5534e-02],
         [-1.2029e-01,  5.5140e-02,  2.4640e-01,  ...,  1.0771e+00,
           7.5527e-01,  1.1561e-01],
         [-1.4317e-01,  2.2736e-02,  1.3421e-01,  ...,  6.9846e-01,
           7.1632e-01, -4.2702e-01],
         ...,
         [ 8.6963e-05, -4.8988e-02,  1.8061e-01,  ...,  4.9731e-01,
           7.4817e-01, -1.9491e-01],
         [ 3.1256e-02,  2.0337e-02,  1.4931e-01,  ...,  3.7829e-01,
           5.2369e-01, -2.7642e-02],
         [ 2.2418e-03,  2.8868e-01,  2.6283e-01,  ...,  8.7499e-01,
           8.7059e-01, -1.3345e-02]], grad_fn=<AddBackward0>),
 'aid': tensor([[-5.0915e-01,  1.8908e-01,  3.7701e-01,  ..., -9.4885e-02,
          -1.2482e-01, -1.2462e-01],
         [-1.0466e+00,  6.7781e-02,  1.0262e+00,  ...,  6.6526e-01,
          -4.6996e-01, -8.5483e-01],
         [-7.3212e-01, -5.6022e-02,  7.7748e-01,  ...,  4.2123e-01,
          -4.1882e-01, -6.3584e-01

In [64]:
model(val_data.x_dict,
      val_data.edge_index_dict,
      val_data['session', 'aid'].edge_label_index)

RuntimeError: ignored

In [68]:
trained_model(val_data.x_dict,
      val_data.edge_index_dict,
      val_data['session', 'aid'].edge_label_index)

IndexError: ignored

### Evaluate a Heterogenous Linklevel GNN

In [57]:
# Define the validation seed edges:
edge_label_index = val_data['session', 'event', 'aid'].edge_label_index
edge_label = val_data['session', 'event', 'aid'].edge_label

val_loader = LinkNeighborLoader(
    data=val_data,  # TODO
    num_neighbors=[20, 10],  # TODO
    #neg_sampling_ratio=0.0,  # TODO
    edge_label_index=(('session', 'event', 'aid'), edge_label_index),
    edge_label=edge_label,
    batch_size=3 * 128,
    shuffle=False,
)

sampled_data = next(iter(val_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

assert sampled_data['session', 'event', 'aid'].edge_label_index.size(1) == 3 * 128
assert sampled_data['session', 'event', 'aid'].edge_label.min() >= 0
assert sampled_data['session', 'event', 'aid'].edge_label.max() <= 2

Sampled mini-batch:
HeteroData(
  session={
    node_id=[3788],
    x=[3788, 32]
  },
  aid={
    x=[1306, 32],
    node_id=[1306]
  },
  (session, event, aid)={
    edge_index=[2, 3535],
    edge_label=[384],
    edge_label_index=[2, 384],
    input_id=[384]
  },
  (aid, rev_event, session)={ edge_index=[2, 3757] }
)


In [58]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

# You need the labels to binarize
labels = [0, 1, 2]

preds = []
ground_truths = []

for sampled_data in tqdm.tqdm(val_loader):
    with torch.no_grad():
        _, pred, ground_truth = test(model=trained_model, data=sampled_data.to(device))
        preds.append(pred)
        ground_truths.append(ground_truth)
        # TODO: Collect predictions and ground-truths and write them into
        # `preds` and `ground_truths`.
        #raise NotImplementedError

pred = torch.cat(preds, dim=0).cpu().numpy()
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()

# Binarize `pred` with shape (n_samples, n_classes)
pred = label_binarize(pred.astype(int), classes=labels)

# Binarize `ground_truth` with shape (n_samples, n_classes)
ground_truth = label_binarize(ground_truth, classes=labels)

#rmse = _
auc = roc_auc_score(ground_truth, pred, multi_class='ovo', average='weighted')
print()
print(f"Validation AUC: {auc:.4f}")
#print('RMSE: ', rmse)

  0%|          | 0/283 [00:00<?, ?it/s]


IndexError: ignored

# Evaluation

In [ ]:
model = train

In [ ]:
del data
gc.collect()

In [ ]:
#trained_model.state_dict()

In [ ]:
!pip install annoy

In [ ]:
%%time
from annoy import AnnoyIndex

index = AnnoyIndex(32, 'angular')

for idx, idx_embedding in enumerate(trained_model.state_dict()['decoder.lin2.weight'].cpu()):
  index.add_item(idx, idx_embedding)

index.build(10)

In [ ]:
#del model, loader, optimizer
#gc.collect()
#torch.cuda.empty_cache()

### Validation/Inference

In [ ]:
# source_idx -> dict{id:idx}
# target_idx -> dict{id:idx}

In [ ]:
test = pl.read_parquet('/content/kaggle/test.parquet')

In [ ]:
test

In [ ]:
source_idx[12899779]

In [ ]:
target_idx[1142000]

In [ ]:
test = test.to_pandas().reset_index(drop=True)

test['session'] = test['session'].map(source_idx)
test['aid'] = test['aid'].map(target_idx)

In [ ]:
test

In [79]:
def evaluate(path,mode="validation",n_neighbors=20):


    test = pl.read_parquet(path)
    test = test.to_pandas().reset_index(drop=True)

    test['session'] = test['session'].map(source_idx)
    test['aid'] = test['aid'].map(target_idx)

    session_types = ['clicks', 'carts', 'orders']
    #test_session_AIDs = test.to_pandas().reset_index(drop=True).groupby('session')['aid'].apply(list)
    #test_session_types = test.to_pandas().reset_index(drop=True).groupby('session')['type'].apply(list)
    test_session_AIDs = test.groupby('session')['aid'].apply(list)
    test_session_types = test.groupby('session')['type'].apply(list)

    del test
    gc.collect()
    labels = []

    type_weight_multipliers = {0: 1, 1: 6, 2: 3}

    for AIDs, types in zip(test_session_AIDs, test_session_types):
        if len(AIDs) >= 20:
            # if we have enough aids (over equals 20) we don't need to look for candidates! we just use the old logic
            weights=np.logspace(0.1,1,len(AIDs),base=2, endpoint=True)-1
            aids_temp=defaultdict(lambda: 0)
            for aid,w,t in zip(AIDs,weights,types): 
                aids_temp[aid]+= w * type_weight_multipliers[t]

            sorted_aids=[k for k, v in sorted(aids_temp.items(), key=lambda item: -item[1])]
            labels.append(sorted_aids[:20])
        else:
            # here we don't have 20 aids to output -- we will use word2vec embeddings to generate candidates!
            AIDs = list(dict.fromkeys(AIDs[::-1]))

            # let's grab the most recent aid
            most_recent_aid = AIDs[0]

            # and look for some neighbors!
            nns = [i for i in index.get_nns_by_item(most_recent_aid, n_neighbors+1)[1:]]


            labels.append((AIDs+nns)[:n_neighbors])

    labels_as_strings = [' '.join([str(l) for l in lls]) for lls in labels]

    predictions = pd.DataFrame(data={'session_type': test_session_AIDs.index, 'labels': labels_as_strings})

    prediction_dfs = []

    for st in session_types:
        modified_predictions = predictions.copy()
        modified_predictions.session_type = modified_predictions.session_type.astype('str') + f'_{st}'
        prediction_dfs.append(modified_predictions)

    sub = pd.concat(prediction_dfs).reset_index(drop=True)
    
    del prediction_dfs, predictions,labels_as_strings, labels, test_session_types,test_session_AIDs
    gc.collect()
    
    if mode=="test":
        sub.to_csv("submission.csv",index=False)
        return sub
    
    else: # validation
        sub['labels_2'] = sub['labels'].apply(lambda x : [int(s) for s in x.split(' ')])
        submission = pd.DataFrame()
        submission['session'] = sub.session_type.apply(lambda x: int(x.split('_')[0]))
        submission['type'] = sub.session_type.apply(lambda x: x.split('_')[1])
        submission['labels'] = sub.labels_2.apply(lambda x : [item for item in x[:] ]) #.apply(lambda x: [int(i) for i in x.split(',')[:20]])
        test_labels = pd.read_parquet('/content/dataset/test_labels.parquet')
        test_labels = test_labels.merge(submission, how='left', on=['session', 'type'])
        del sub,submission
        gc.collect()
        gc.collect()
        test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
        test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
        recall_per_type = test_labels.groupby(['type'])['hits'].sum() / test_labels.groupby(['type'])['gt_count'].sum() 
        score = (recall_per_type * pd.Series({'clicks': 0.1, 'carts': 0.30, 'orders': 0.60})).sum()

        return score

In [ ]:
%debug

### Validation

In [80]:
path = '/content/kaggle/test.parquet'
validation_score = evaluate(path,mode="validation",n_neighbors=20)

TypeError: ignored

In [81]:
%debug

> <ipython-input-79-e576a05d6733>(76)<lambda>()
     74         gc.collect()
     75         gc.collect()
---> 76         test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
     77         test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
     78         recall_per_type = test_labels.groupby(['type'])['hits'].sum() / test_labels.groupby(['type'])['gt_count'].sum()

ipdb> test_labels
*** NameError: name 'test_labels' is not defined
ipdb> up
> /usr/local/lib/python3.8/dist-packages/pandas/core/apply.py(828)apply_series_generator()
    826             for i, v in enumerate(series_gen):
    827                 # ignore SettingWithCopy here in case the user mutates
--> 828                 results[i] = self.f(v)
    829                 if isinstance(results[i], ABCSeries):
    830                     # If we have a view on v, we need to make a copy because

ipdb> up
> /usr/local/lib/python3.8/dist-packages/


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.8/bdb.py", line 359, in set_quit
    sys.settrace(None)



In [83]:
print(validation_score)

NameError: ignored

# Submission

Submission File
For each `session` id and `type` combination in the test set, you must predict the `aid` values in the `label` column, which is space delimited. You can predict up to 20 `aid` values per row. The file should contain a header and have the following format:

```
session_type,labels
12906577_clicks,135193 129431 119318 ...
12906577_carts,135193 129431 119318 ...
12906577_orders,135193 129431 119318 ...
12906578_clicks, 135193 129431 119318 ...
etc.
```